In [1]:
!pip install -q -U transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/bert/bert

/content/drive/MyDrive/bert/bert


## 1. Load Dataset

In [6]:
import pandas as pd

data_train = pd.read_csv('ntc_scv_train.csv')
data_test = pd.read_csv('ntc_scv_test.csv')

## 2. Preprocessing

In [7]:
import pandas as pd
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

# Tên mô hình
model_name = "distilbert-base-uncased"

# Khởi tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Đặt chiều dài tối đa của chuỗi
max_seq_length = 100
max_seq_length = min(max_seq_length, tokenizer.model_max_length)

# Hàm xử lý trước (preprocessing) cho DataFrame
def preprocess_function(row):
    result = tokenizer(
        row["preprocessed_sentence"],  # Đảm bảo cột dữ liệu đúng tên
        padding="max_length",  # Padding đến chiều dài tối đa
        max_length=max_seq_length,  # Đặt chiều dài tối đa
        truncation=True  # Cắt bớt nếu dài hơn max_length
    )

    # Thêm nhãn (label) vào kết quả tokenized
    result["label"] = row["label"]  # Đảm bảo cột 'label' đúng tên

    return result


# Áp dụng hàm xử lý cho tất cả các dòng trong DataFrame train
processed_train_data = data_train.apply(preprocess_function, axis=1)

# Chuyển kết quả thành DataFrame cho dữ liệu train
processed_train_data = pd.DataFrame(processed_train_data.tolist(), columns=["input_ids", "attention_mask", "label"])

# Áp dụng hàm xử lý cho tất cả các dòng trong DataFrame test
processed_test_data = data_test.apply(preprocess_function, axis=1)

# Chuyển kết quả thành DataFrame cho dữ liệu test
processed_test_data = pd.DataFrame(processed_test_data.tolist(), columns=["input_ids", "attention_mask", "label"])

# Kiểm tra kết quả
print("Processed Train Data:")
print(processed_train_data.head())

print("Processed Test Data:")
print(processed_test_data.head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Processed Train Data:
                                           input_ids  \
0  [101, 12731, 2050, 6865, 18699, 2080, 8418, 25...   
1  [101, 19538, 25175, 4638, 1999, 24110, 29349, ...   
2  [101, 24110, 1047, 19991, 1047, 6806, 5199, 18...   
3  [101, 19538, 2000, 2072, 14071, 14163, 2050, 2...   
4  [101, 2771, 18699, 1102, 2072, 24209, 2050, 10...   

                                      attention_mask  label  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...      1  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...      1  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...      1  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...      1  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...      1  
Processed Test Data:
                                           input_ids  \
0  [101, 2474, 24110, 21122, 15775, 5199, 21110, ...   
1  [101, 7221, 2232, 17895, 2078, 24110, 1102, 13...   
2  [101, 1102, 4710, 2474, 2053, 2072, 22794, 105...   
3  [101, 12256, 8945, 20

In [8]:
from datasets import Dataset, DatasetDict

# Chuyển đổi dữ liệu đã xử lý thành Dataset của Hugging Face
train_dataset = Dataset.from_pandas(processed_train_data)
test_dataset = Dataset.from_pandas(processed_test_data)

# Tạo DatasetDict để chứa cả train và test dataset
processed_dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

# Kiểm tra dữ liệu
print(processed_dataset["train"].features)


{'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'label': Value(dtype='int64', id=None)}


In [9]:
from transformers import AutoConfig, AutoModelForSequenceClassification

# Số lượng nhãn cho bài toán phân loại
num_labels = 2

# Tải cấu hình và mô hình
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    finetuning_task="text-classification"
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config
)

# Kiểm tra mô hình
print(model)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [10]:
import numpy as np
import evaluate

# Tải chỉ số accuracy
metric = evaluate.load("accuracy")

# Hàm tính độ chính xác
def compute_metrics(eval_pred):
    # Lấy dự đoán và nhãn
    predictions, labels = eval_pred

    # Nếu mô hình trả về logits (đầu ra chưa qua softmax), chọn nhãn có xác suất cao nhất
    predictions = np.argmax(predictions, axis=1)  # Chọn nhãn có xác suất cao nhất

    # Tính toán độ chính xác
    result = metric.compute(predictions=predictions, references=labels)

    return result


In [11]:
from transformers import Trainer, TrainingArguments

# Định nghĩa các tham số huấn luyện
training_args = TrainingArguments(
    output_dir="model/checkpoints",         # Đường dẫn lưu checkpoint
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=10,
    evaluation_strategy="epoch",            # Đánh giá sau mỗi epoch
    save_strategy="epoch",                  # Lưu checkpoint sau mỗi epoch
    save_total_limit=3,                     # Chỉ lưu lại 3 checkpoint mới nhất
    load_best_model_at_end=True,            # Tải mô hình tốt nhất sau khi huấn luyện
    metric_for_best_model="accuracy",       # Tiêu chí lựa chọn mô hình tốt nhất
    greater_is_better=True,                 # Giá trị cao hơn là tốt hơn
)

# Tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# Huấn luyện mô hình
trainer.train()

# Lưu mô hình tốt nhất và tokenizer
trainer.save_model("model/best_model")
tokenizer.save_pretrained("model/tokenizer")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-1efc7be072ea>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.438649,0.805100
2,No log,0.440512,0.803900


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.438649,0.805100
2,No log,0.440512,0.803900
3,0.451300,0.392478,0.826800
4,0.451300,0.386949,0.836900
5,0.336600,0.416878,0.836400
6,0.336600,0.406579,0.832200
7,0.273300,0.405192,0.839100
8,0.273300,0.429220,0.835200
9,0.232400,0.447288,0.835500
10,0.232400,0.446807,0.836600


('model/tokenizer/tokenizer_config.json',
 'model/tokenizer/special_tokens_map.json',
 'model/tokenizer/vocab.txt',
 'model/tokenizer/added_tokens.json',
 'model/tokenizer/tokenizer.json')